In [1]:
%load_ext autoreload
%autoreload 2
import sys
import gc

from multiprocessing import cpu_count, Pool
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn
%matplotlib inline

In [2]:
torch.manual_seed(1)
torch.cuda.set_device(0)

AttributeError: module 'torch._C' has no attribute '_cuda_setDevice'

In [3]:
batch_size = 256

In [4]:
pred_len = 60

In [5]:
base_dir = '../data/'

In [6]:
train_df = pd.read_csv(base_dir+'train_1.csv').fillna(0)

In [7]:
X = train_df.drop('Page', axis=1).values

In [9]:
X, scaler = rnn.scale_values(X)

In [11]:
trainloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-2*pred_len,:]).float(),
        torch.from_numpy(X[:,-2*pred_len:-pred_len,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [13]:
valloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [25]:
testloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:,:]).float(),
        torch.zeros(X.shape[0])
    ),
    batch_size=batch_size, shuffle=False
)

In [30]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func)

In [223]:
optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=10)